In [3]:
import sys
sys.path.append('..')

from layersdk import dataset, model, Layer, File, Dataset, SQL, assert_unique, \
    assert_not_null, assert_valid_values, assert_true

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

In [9]:
def clean_sex(sex):
    result = 0
    if sex == "female": result = 0
    elif sex == "male": result = 1
    return result


def clean_age(data):
    age = data[0]
    pclass = data[1]
    if pd.isnull(age):
        if pclass == 1: return 37
        elif pclass == 2: return 29
        else: return 24
    else:
        return age

@assert_valid_values('Sex', [1, 0])
@dataset('passengers', depends=[File('titanic.csv')])
def read_dataset():
    df = pd.read_csv("titanic.csv")
    df['Sex'] = df['Sex'].apply(clean_sex)
    df['Age'] = df[['Age', 'Pclass']].apply(clean_age, axis=1)
    df = df.drop(["PassengerId", "Name", "Cabin", "Ticket", "Embarked"], axis=1)
    return df

@model(name='survival_model')
def train():
    df = layer.get_dataset("passengers").to_pandas()
    X = df.drop(["Survived"], axis=1)
    y = df["Survived"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                        random_state=42)
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(X_train, y_train)
    y_pred = random_forest.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    layer.log_metric("accuracy", acc)
    return random_forest

layer = Layer(project_name="ltv_project", environment='requirements.txt')
# layer.run([read_dataset, train])
# train()
read_dataset()

	Test SUCCESS: assert_valid_values('Sex', [1, 0])


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000
887,1,1,0,19.0,0,0,30.0000
888,0,3,0,24.0,1,2,23.4500
889,1,1,1,26.0,0,0,30.0000
